In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import List, Callable, Any
from json import JSONDecoder
from decimal import Decimal
from datetime import datetime, timedelta
from pydantic import parse_raw_as
from coinbase.candle_sync import Coin
import psycopg
from psycopg import sql
from coinbase.db import settings
from coinbase.db import models

In [3]:
config = settings.Settings()
end = datetime(2021, 11, 30, 12, 0)
start = end - timedelta(seconds=(300*60*1))

In [4]:
api = Coin("ADA-USD")
data = api.history(60, start, end)

In [5]:
coin = models.CoinHistory.from_coinbase(data[0].text)

In [9]:
#sql_str = models.CoinHistory.create_table_str("ADA-USD")
sql_str = models.CoinHistory.insert_str("ADA-USD")
with psycopg.connect(config.conn_str) as conn:
    with conn.cursor() as cur:
        cur.execute(sql_str.as_string(conn), *[row.as_tuple() for row in coin])

ProgrammingError: the query has 6 placeholders but 300 parameters were passed

In [11]:
[row.as_tuple() for row in coin]

300

In [11]:
coin_id = "ADA-USD"
price_precision = 10
price_scale = 5
sql_str = sql.SQL("""
            CREATE TABLE {table} (
                timestamp TIMESTAMP PRIMARY KEY,
                low {price_type},
                high {price_type},
                open {price_type},
                close {price_type},
                volumne DOUBLE PRECISION
                )
            """).format(
                table = sql.Identifier(f'{coin_id}_history'),
                price_type = sql.SQL(f'NUMERIC({price_precision},{price_scale})')
            )
with psycopg.connect(config.conn_str) as conn:
    print(sql_str.as_string(conn))


            CREATE TABLE "ADA-USD_history" (
                timestamp TIMESTAMP PRIMARY KEY,
                low NUMERIC(10,5),
                high NUMERIC(10,5),
                open NUMERIC(10,5),
                close NUMERIC(10,5),
                volumne DOUBLE PRECISION
                )
            


In [12]:
with psycopg.connect(config.conn_str) as conn:
    with conn.cursor() as cur:
        cur.execute(sql_str)

In [13]:
type(sql_str)

psycopg.sql.Composed